In [49]:
import whisper
import datasets
import pydub
import soundfile
import pandas as pd
import librosa
from pydub import AudioSegment

In [4]:
dataset = datasets.load_dataset("CAiRE/ASCEND")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

C:\Users\saleh\.cache\huggingface\datasets\downloads\c0c90a40ffbbe502bf0aa0911c0e2f2afd3837249582355c0b3f798364d8f766


Generating test split: 0 examples [00:00, ? examples/s]

C:\Users\saleh\.cache\huggingface\datasets\downloads\e0245f18a1c0d6208d163a72d77fa3b36dd8082671e3acb1bd0a6436501777b3


Generating validation split: 0 examples [00:00, ? examples/s]

C:\Users\saleh\.cache\huggingface\datasets\downloads\82d074c3031ceb89eb3ee2f120e92edd4eb64530261da4b1ee8a04b10aef0928


In [11]:
# Function to separate mp3 file by timestamp
set = pd.DataFrame(dataset['validation'])
display(set)

,id,path,audio,transcription,duration,language,original_speaker_id,session_id,topic
0,00000,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,嗯,0.820,zh,20,4,technology
1,00001,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,小朋友我回想一下when i was in elementary school,4.600,mixed,20,4,technology
2,00002,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,like year three,1.740,en,20,4,technology
3,00003,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,i have a phone but it's not a smart phone at t...,3.500,en,20,4,technology
4,00004,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,就是其实主要的功能就是打电话然后跟parents联系,6.020,mixed,20,4,technology
...,...,...,...,...,...,...,...,...,...
1125,01125,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,嗯我也觉得 [UNK] 就是那个L G三的那个gym every time I go the...,6.900,mixed,12,4,sports
1126,01126,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,when I go to the [UNK] nobody,2.990,en,12,4,sports
1127,01127,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,所以非常非常非常对比非常鲜明所以我就不去健身房那个L G三的健身房,5.710,mixed,12,4,sports
1128,01128,C:\Users\saleh\.cache\huggingface\datasets\dow...,{'path': 'C:\Users\saleh\.cache\huggingface\da...,嗯很大一部分原因就是像你说的是因为水土不服还有,4.375,zh,7,3,education


In [71]:
path = set.at[10, 'path']
print(path)

C:\Users\saleh\.cache\huggingface\datasets\downloads\extracted\f7ac516b8b52e26639eabec77d98278b293cb5a5579773126b8901eb92fb8307\waves/ses4_spk20_L1616_49.6100_9.3600.wav


In [72]:
total_wav = AudioSegment.from_wav(path)
total_wav_file = total_wav.export("total_wav.wav", format="wav")
duration = len(total_wav)
model = whisper.load_model("base")

def split_audio(start, end):
    split = start+(end-start)/2
    first_half = total_wav[start:split]
    second_half = total_wav[split:end]
    first_half.export("first_half.wav", format="wav")
    second_half.export("second_half.wav", format="wav")

def audio_interval(start, end):
    interval = total_wav[start:end]
    interval.export("interval.wav", format="wav")

def get_highest_whisper_entry(file):
    audio = whisper.load_audio(file)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
    sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]
    return sorted_probs[0]

# my_dict = {}
# def binary_search(file):
#     if get_highest_whisper_entry(file)['value'] > 0.9:
#         my_dict[0] = get_highest_whisper_entry(file)['key']
#         return my_dict
#     start = 0
#     end = duration
#     while(end-start > 500):
#         split_audio(start, start+(end-start)/2)
#         split_audio(start+(end-start)/2, end)

def create_dict_from_intervals():
    start = 0
    my_dict = {}
    while(start<duration):
        audio_interval(start, start+2000)
        file = "interval.wav"
        print(get_highest_whisper_entry(file)['value'])
        if get_highest_whisper_entry(file)['value'] < 0.90:
            for i in range(4):
                audio_interval(start+(i)*500, start+i*500)
                my_dict[start+(i)*500] = get_highest_whisper_entry(file)['key']
        else:
            my_dict[start] = get_highest_whisper_entry(file)['key']
        start += 2000
    return my_dict

print(create_dict_from_intervals())

0.9472328424453735
0.6645970344543457
0.9411177635192871
0.8626769781112671
0.8410094976425171
{0: 'zh', 2000: 'en', 2500: 'en', 3000: 'en', 3500: 'en', 4000: 'zh', 6000: 'en', 6500: 'en', 7000: 'en', 7500: 'en', 8000: 'en', 8500: 'en', 9000: 'en', 9500: 'en'}


In [ ]:
model = whisper.load_model("base")

In [73]:
audio = whisper.load_audio("total_wav.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)

_, probs = model.detect_language(mel)

sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)

# Creating a new list with the sorted key-value pairs
sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]

print(sorted_probs)



audio = whisper.load_audio("first_half.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)

_, probs = model.detect_language(mel)

sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)

# Creating a new list with the sorted key-value pairs
sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]

print(sorted_probs)



audio = whisper.load_audio("second_half.wav")
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)

_, probs = model.detect_language(mel)

sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)

# Creating a new list with the sorted key-value pairs
sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]

print(sorted_probs)

[{'key': 'zh', 'value': 0.9570052027702332}, {'key': 'en', 'value': 0.019680246710777283}, {'key': 'ko', 'value': 0.012031739577651024}, {'key': 'ja', 'value': 0.003600840223953128}, {'key': 'th', 'value': 0.0011827453272417188}, {'key': 'de', 'value': 0.0007861490012146533}, {'key': 'ru', 'value': 0.0006601502536796033}, {'key': 'fr', 'value': 0.0006259232177399099}, {'key': 'nn', 'value': 0.0005809604772366583}, {'key': 'ms', 'value': 0.00046438383287750185}, {'key': 'jw', 'value': 0.00042146912892349064}, {'key': 'vi', 'value': 0.00041443025111220777}, {'key': 'haw', 'value': 0.00028859914164058864}, {'key': 'la', 'value': 0.00023940544633660465}, {'key': 'pt', 'value': 0.00023817573674023151}, {'key': 'es', 'value': 0.00017488804587628692}, {'key': 'cy', 'value': 0.0001692837249720469}, {'key': 'pl', 'value': 0.00016809391672722995}, {'key': 'it', 'value': 0.0001560395467095077}, {'key': 'id', 'value': 0.000152472872287035}, {'key': 'tr', 'value': 0.00014974210353102535}, {'key': '

In [ ]:
_, probs = model.detect_language(mel)

# Sorting the dictionary items by values in descending order
sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)

# Creating a new list with the sorted key-value pairs
sorted_probs = [{'key': key, 'value': value} for key, value in sorted_probs]

print(sorted_probs)

In [61]:
options = whisper.DecodingOptions(task='transcribe', fp16=False)
result = whisper.decode(model, mel, options)
print(result.text)

都看到小朋友們其實有受傷的會拿著 Smartphone在玩


In [62]:
options = whisper.DecodingOptions(task='translate', fp16=False)
result = whisper.decode(model, mel, options)
print(result.text)

I can see that the small ones are actually very small. The small ones will be used by SmartForn. And then play.
